In [1]:
# Import libraries and dependencies
import os
import shutil
import csv
import subprocess

In [2]:
# Get working directory
os.chdir("/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run")
dir=os.getcwd()

# WOX5 sequence
fasta_sequence="MSFSVKGRSLRGNNNGGTGTKCGRWNPTVEQLKILTDLFRAGLRTPTTDQIQKISTELSFYGKIESKNVFYWFQNHKARERQKRRKISIDFDHHHHQPSTRDVFEISEEDCQEEEKVIETLQLFPVNSFEDSNSKVDKMRARGNNQYREYIRETTTTSFSPYSSCGAEMEHPPPLDLRLSFL"

# Conditions
pH=7.0
temp=298
ionic=0.15
PAE_cut=5 # Default
NR=6 # 6 replicas

# Od regions are defined as regions of more than 3 residues with pLDDT>0.75.
ordered_domains = {'od1': [23, 83]}
disordered_domains={'dd1': [1, 22],'dd2':[84, 182]}

# AF input created in AF-distance map prediction and used in AF-MI
pdb_af='WOX5_7372f_unrelaxed_rank_001_alphafold2_ptm_model_1_seed_000.pdb'
json_af='WOX5_7372f_predicted_aligned_error_v1.json'
npy_af='alphafold2_ptm_model_1_seed_000_prob_distributions.npy'
mean_af='alphafold2_ptm_model_1_seed_000_mean.csv'

# Copy AF data
shutil.copy2(dir+"/../AF_DATA_WOX5/"+pdb_af, dir+"/pdb_af.pdb")
shutil.copy2(dir+"/../AF_DATA_WOX5/"+json_af, dir+"/pae.json")
shutil.copy2(dir+"/../AF_DATA_WOX5/WOX5_7372f_distmat/"+mean_af, dir+"/mean_af.csv")
shutil.copy2(dir+"/../AF_DATA_WOX5/WOX5_7372f_distmat/"+npy_af, dir+"/prob.npy")

'/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run/prob.npy'

In [3]:
# Write sequence in a new file
f = open("sequence.dat", "w")
f.write(fasta_sequence)
f.close()

# Write the csv files
with open('ordered_domains.csv', 'w') as f:
    w = csv.DictWriter(f, ordered_domains.keys())
    w.writeheader()
    w.writerow(ordered_domains)
with open('disordered_domains.csv', 'w') as f:
    w = csv.DictWriter(f, disordered_domains.keys())
    w.writeheader()
    w.writerow(disordered_domains)

# Copy OPENMM calvados forcefield files
shutil.copy2(dir+"/../scripts_prep/gen_xml_and_constraints.py", dir)
shutil.copy2(dir+"/../scripts_prep/residues.csv", dir)

path_gen_xml = dir+'/gen_xml_and_constraints.py sequence.dat '+str(pH)+' '+str(temp)+' '+str(ionic)
print(path_gen_xml)
os.system(f'python {path_gen_xml}')

/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run/gen_xml_and_constraints.py sequence.dat 7.0 298 0.15


/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run/gen_xml_and_constraints.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.09090909090909091' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r.loc['H','q'] = 1. / ( 1 + 10**(pH-6) )


0

In [4]:
# Copy and run the prep script that makes the plumed file
shutil.copy2(dir+"/../scripts_prep/make_plumed_distmat.py", dir)
subprocess.run(['python', str(dir)+'/make_plumed_distmat.py', 'sequence.dat',str(PAE_cut), '0.2'], capture_output=True, text=True)

CompletedProcess(args=['python', '/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run/make_plumed_distmat.py', 'sequence.dat', '5', '0.2'], returncode=0, stdout="5.0 0.2\n[[23, 83]]\n        od1\n0  [23, 83]\n[[1, 22], [84, 182]]\n       dd1        dd2\n0  [1, 22]  [84, 182]\n1 4\n1 5\n1 6\n1 7\n1 8\n1 9\n1 10\n1 11\n1 12\n1 13\n1 14\n1 15\n1 16\n1 17\n1 18\n1 19\n1 20\n1 21\n1 22\n1 23\n1 24\n1 25\n1 26\n1 27\n1 28\n1 29\n1 30\n1 31\n1 32\n1 33\n1 34\n1 35\n1 36\n1 37\n1 38\n1 39\n1 40\n1 41\n1 42\n1 43\n1 44\n1 45\n1 46\n1 47\n1 48\n1 49\n1 50\n1 51\n1 52\n1 53\n1 54\n1 55\n1 56\n1 57\n1 58\n1 59\n1 60\n1 61\n1 62\n1 63\n1 64\n1 65\n1 66\n1 67\n1 68\n1 69\n1 70\n1 71\n1 72\n1 73\n1 74\n1 75\n1 76\n1 77\n1 78\n1 79\n1 80\n1 81\n1 82\n1 83\n1 84\n1 85\n1 86\n1 87\n1 88\n1 89\n1 90\n1 91\n1 92\n1 93\n1 94\n1 95\n1 96\n1 97\n1 98\n1 99\n1 100\n1 101\n1 102\n1 103\n1 104\n1 105\n1 106\n1 107\n1 108\n1 109\n1 110\n1 111\n1 112\n1 113\n1 114\n1 115\n

In [5]:
# Make the plumed_analysis.dat file
shutil.copy2(dir+"/../scripts_prep/make_plumed_analysis.py", dir)
path_gen_analysis = dir+'/make_plumed_analysis.py sequence.dat'
os.system(f'python {path_gen_analysis}')

[[23, 83]]
        od1
0  [23, 83]
[[1, 22], [84, 182]]
       dd1        dd2
0  [1, 22]  [84, 182]
key dis {'od1': [[23, 83]]}
key dis {'dd1': [[1, 22]], 'dd2': [[84, 182]]}


0

In [6]:
shutil.copy2(dir+"/../scripts_prep/simulate_em.py", dir)
simulate_em = dir+'/simulate_em.py '+str(pH)+' '+str(temp)
os.system(f'python {simulate_em}')

<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
/lustre/BIF/nobackup/perez070/scRNA-seq_PLT1-WOX5/github_repositories/AlphaFold-IDP/prep_run/simulate_em.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.09090909090909091' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  r.loc['H','q'] = 1. / ( 1 + 10**(pH-6) )


0

In [7]:
shutil.copy2(dir+"/../scripts_prep/simulate.py", dir)
!mpirun -np {NR} python simulate.py {pH} {temp}

<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 